In [ ]:
from ndtools import staged_max_flow as smf

from pathlib import Path
import json

from tsum import tsum
import torch

# Reliability analysis of benchmark gas supply plant

## Case 1: Original system

### Load data

In [ ]:
DATASET = Path(r"data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))


In [ ]:
def s_fun(comps_st):
    flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, nodes, edges, probs_dict, target_flow = 90.0 )

    sys_st = 1 if sys_st_str == 's' else 0
    return flow, sys_st, min_comp_state

row_names = list(probs_dict.keys())
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

### Get rules for system event

In [ ]:
sys_surv_st = 1 

result = tsum.run_rule_extraction_by_mcs(
    # Problem-specific callables / data
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    sys_surv_st=sys_surv_st,
    unk_prob_thres = 1e-5,
    output_dir="tsum_res"
) 

## Case 2: Addition Importance Measure

### Get rules for system event with each component added

In [ ]:
for x in probs_dict.keys():
    print(f"Adding component: {x}")

    new_nodes, new_edges, new_probs = smf.add_a_component(x, nodes, edges, probs_dict)

    def s_fun(comps_st):
        flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, new_nodes, new_edges, new_probs, target_flow = 90.0 )

        sys_st = 1 if sys_st_str == 's' else 0
        return flow, sys_st, min_comp_state
    
    row_names = list(new_probs.keys())
    n_state = 2  # binary states: 0, 1
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    probs = [[new_probs[n]['0']['p'], new_probs[n]['1']['p']] for n in row_names]
    probs = torch.tensor(probs, dtype=torch.float32, device=device)

    # get rules
    sys_surv_st = 1
    
    result = tsum.run_rule_extraction_by_mcs(
        # Problem-specific callables / data
        sfun=s_fun,
        probs=probs,
        row_names=row_names,
        n_state=n_state,
        sys_surv_st=sys_surv_st,
        unk_prob_thres = 1e-5,
        output_dir=rf"tsum_res_aim/{x}"
    ) 

## Case 3: Deactivation Importance Measure

### Get rules for system event with each component deactivated

In [ ]:
for x in probs_dict.keys():
    print(f"Adding component: {x}")

    new_nodes, new_edges, new_probs = smf.deactivate_a_component(x, nodes, edges, probs_dict)

    def s_fun(comps_st):
        flow, sys_st_str, min_comp_state = smf.sys_fun( comps_st, new_nodes, new_edges, new_probs, target_flow = 90.0 )

        sys_st = 1 if sys_st_str == 's' else 0
        return flow, sys_st, min_comp_state
    
    row_names = list(new_probs.keys())
    n_state = 2  # binary states: 0, 1
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    probs = [[new_probs[n]['0']['p'], new_probs[n]['1']['p']] for n in row_names]
    probs = torch.tensor(probs, dtype=torch.float32, device=device)

    # get rules
    sys_surv_st = 1
    
    result = tsum.run_rule_extraction_by_mcs(
        # Problem-specific callables / data
        sfun=s_fun,
        probs=probs,
        row_names=row_names,
        n_state=n_state,
        sys_surv_st=sys_surv_st,
        unk_prob_thres = 1e-5,
        output_dir=rf"tsum_res_dim/{x}"
    ) 